This exercise will require you to pull some data from the Quandl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import collections
import json

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?start_date=2018-11-10&end_date=2018-11-11&api_key=' + API_KEY)

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
afx_x = json.loads(r.text)
afx_x

{'dataset': {'id': 10095370,
  'dataset_code': 'AFX_X',
  'database_code': 'FSE',
  'name': 'Carl Zeiss Meditec (AFX_X)',
  'description': 'Stock Prices for Carl Zeiss Meditec (AFX) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
  'refreshed_at': '2018-11-20T22:31:42.439Z',
  'newest_available_date': '2018-11-20',
  'oldest_available_date': '2000-06-07',
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'frequency': 'daily',
  'type': 'Time Series',
  'premium': False,
  'limit': None,
  'transform': None,
  'column_index': None,
  'start_date': '2018-11-10',
  'end_date': '2018-11-11',
  'data': [],
  'collapse': None,
  'order': None,
  'database_id': 6129}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

# 1. AFX_X 2017 data 

In [5]:
r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2018-01-01&api_key=' + API_KEY)
# pull 2017 data from quandl

# 2. JSON to python dictionary

Convert JSON object into python dictionary using json.loads()

In [6]:
afx_x = json.loads(r.text)
type(afx_x)

dict

# 3. highest & lowest opening prices 2017

In [7]:
afx = afx_x['dataset'] # extract relevant data
keys = afx['column_names']; # create a list of column names for dict keys 
data = afx['data']

The data we are interested in, 'data', is formatted as a list of lists. We can transpose the data, whose rows correspond to information for a given day, into columns of data associated with their respective variables.

In [8]:
afx_data = [[row[i] for row in data] for i in range(len(data[0]))]

In [9]:
i_open = [i for i in range(len(afx_data[1])) if afx_data[1][i] is not None] # return non-None indices
open_sel = [afx_data[1][i] for i in i_open] # select non-None values from the list of opening prices
Highest_2017 = max(open_sel); Lowest_2017 = min(open_sel)
print(Highest_2017, Lowest_2017)

53.11 34.0


The highest and lowest opening prices in 2017 were 53.11 and 34.0, respectively.

# 4. Largest change in any one day (high & low prices)

In [10]:
delta1 = [a - b for a, b in zip(afx_data[2],afx_data[3])]
max(delta1)

2.8100000000000023

The largest change in any one day, based off of high and low prices, is 2.81.

# 5. Largest change between any two days (based on closing price)

In [11]:
delta2 = [c - d for c, d in zip(afx_data[4][:-1], afx_data[4][1:])]
max(delta2)

1.7199999999999989

The largest change between any two days, based on closing price, is 1.72.

# 6. Average daily trading volume 2017

In [12]:
def avg(n):
    return float(sum(n)) / max(len(n),1)

In [13]:
avg(afx_data[6])

89124.33725490196

The average daily trading volume of AFX_X in 2017 was 89,124.

# 7. Median trading volume 2017

In [14]:
def med(v):
    k = len(v)
    if k < 1:
        return None
    if k % 2 == 1:
        return sorted(v)[k//2]
    else:
        return sum(sorted(v)[k//2-1:k//2+1])/2

In [15]:
med(afx_data[6])

76286.0

The median trading volume of AFX_X in 2017 was 76,286.